In [2]:
import BendrModels
import torch 

In [3]:
CONTEXT_PATH = "/itet-stor/wolflu/net_scratch/projects/BENDR/pretrained/contextualizer.pt"
ENCODER_PATH = "/itet-stor/wolflu/net_scratch/projects/BENDR/pretrained/encoder.pt"

In [5]:
# convolutional encoder
conv_encoder = BendrModels.ConvEncoderBENDR(in_features=20)
sum(p.numel() for p in conv_encoder.parameters() if p.requires_grad)

conv_encoder.load(ENCODER_PATH, cpu=True)

In [6]:
conv_encoder

ConvEncoderBENDR(
  (encoder): Sequential(
    (Encoder_0): Sequential(
      (0): Conv1d(20, 512, kernel_size=(3,), stride=(3,), padding=(1,))
      (1): Dropout2d(p=0.0, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='none')
    )
    (Encoder_1): Sequential(
      (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): Dropout2d(p=0.0, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='none')
    )
    (Encoder_2): Sequential(
      (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): Dropout2d(p=0.0, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='none')
    )
    (Encoder_3): Sequential(
      (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): Dropout2d(p=0.0, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='non

In [19]:
# transformer encoder
contextualizer = BendrModels.BENDRContextualizer(in_features=256)
sum(p.numel() for p in contextualizer.parameters() if p.requires_grad)

57226809

In [6]:
#contextualizer.load(CONTEXT_PATH, cpu=True)

In [26]:
t = torch.randn(16, 128, 500)

In [27]:
conv_encoder(t).shape

torch.Size([16, 256, 6])

In [ ]:
process = BendrModels.BendingCollegeWav2Vec(conv_encoder, contextualizer)